In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
from __future__ import print_function
import keras
from tensorflow.keras.utils import to_categorical #using ternsorflow to inport keras.utils (deu pau pra importar só do Keras (ainda não sei pq)
import os
from keras.preprocessing.image import ImageDataGenerator, load_img

In [ ]:
from google.colab import drive
drive.mount('/content/drive/',  force_remount=True)

TRAIN MODEL

In [ ]:
from tensorflow.keras.applications import VGG16 # same keras bug here (não consegui usar "from keras.applications import VGG16" acho que isso obriga a usar o tensorflow no background)

vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(256, 256, 3))

In [ ]:
#Base directory

train_dir = '/content/drive/My Drive/cancer_detection/paper/model_2/patch_train'
validation_dir = '/content/drive/My Drive/cancer_detection/paper/model_2/patch_val'
test_dir = '/content/drive/My Drive/cancer_detection/paper/model_2/patch_test'

In [ ]:
vgg_conv.summary()

In [ ]:
# this is the augmentation configuration we will use for training
data_augmentation = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True)

batch_size = 20
num_train = 206

train_patches = np.zeros(shape=(num_train, 8, 8, 512)) # I have to study how does dimensions works ---> DISCUSSING POINT <---
train_labels = np.zeros(shape=(num_train,2))

train_generator = data_augmentation.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical')

i = 0
for inputs_batch, labels_batch in train_generator:
    patches_batch = vgg_conv.predict(inputs_batch)
    train_patches[i * batch_size : (i + 1) * batch_size] = patches_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= num_train: # to stop and to do not generate a loop
        break
        
train_patches = np.reshape(train_patches, (num_train, 8 * 8 * 512))

In [ ]:
num_val = 55

data_augmentation = ImageDataGenerator(rescale=1./255)

validation_patches = np.zeros(shape=(num_val, 8, 8, 512))
validation_labels = np.zeros(shape=(num_val,2))

validation_generator = data_augmentation.flow_from_directory(
    validation_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical')

i = 0
for inputs_batch, labels_batch in validation_generator:
    patches_batch = vgg_conv.predict(inputs_batch)
    validation_patches[i * batch_size : (i + 1) * batch_size] = patches_batch
    validation_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= num_val:
        break

validation_patches = np.reshape(validation_patches, (num_val, 8 * 8 * 512))

In [ ]:
num_test = 57

test_patches = np.zeros(shape=(num_test, 8, 8, 512))
test_labels = np.zeros(shape=(num_test,2))

data_augmentation = data_augmentation = ImageDataGenerator(rescale=1./255)

test_generator = data_augmentation.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical')

i = 0
for inputs_batch, labels_batch in test_generator:
    patches_batch = vgg_conv.predict(inputs_batch)
    test_patches[i * batch_size : (i + 1) * batch_size] = patches_batch
    test_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= num_test:
        break

test_patches = np.reshape(test_patches, (num_test, 8 * 8 * 512))

TRAINING

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=8 * 8 * 512)) # I need to study how does dimensions works. Define best architecture ---> DISCUSSING POINT <---
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_patches,
                    train_labels,
                    epochs=20,
                    batch_size=batch_size,
                    validation_data=(validation_patches,validation_labels))

SAVE MODEL

In [ ]:
# Save the Model

model_dir = '/content/drive/My Drive/cancer_detection/paper/model_2'

model_name = 'model_vgg_cancer_vs2.h5' 
#model_vgg_cancer_all_vs1.h5 -> all methods was implemented
#model_vgg_cancer_vs2.h5 -> stain norm was not implemented

model.save(os.path.join(model_dir, model_name))

In [ ]:
# Recreating same model
new_model = keras.models.load_model(os.path.join(model_dir, model_name))
new_model.summary()

In [ ]:
loss, acc = new_model.evaluate(validation_patches,validation_labels)
print("Model validation accuracy: {:5.2f}%".format(100*acc))

loss, acc = new_model.evaluate(test_patches,test_labels)
print("Model test accuracy: {:5.2f}%".format(100*acc))

In [ ]:
# Plot the accuracy and loss curves ---> nedd to undertand better this graph
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()